In [ ]:
# ============================================================
# Shadow Dexterous Hand + Crosley Alarm Clock
# 5 grasp trials, fixed camera, no debug prints
# ============================================================

import os
import time
import mujoco
import mujoco.viewer
import numpy as np

# -------------------------------
# CONFIG (ТВОИ ПУТИ)
# -------------------------------

ASSETS_DIR = r"C:\Users\rad\anaconda3\envs\dexgrasp\Lib\site-packages\gymnasium_robotics\envs\assets\hand"

XML_PATH = r"C:\Users\rad\anaconda3\envs\dexgrasp\Lib\site-packages\gymnasium_robotics\envs\assets\hand\hand_manipulate_clock.xml"

NUM_TRIALS  = 5
OPEN_STEPS  = 40
CLOSE_STEPS = 120
HOLD_STEPS  = 200

print("Assets dir:", ASSETS_DIR)
print("Clock scene XML exists:", os.path.exists(XML_PATH))

# -------------------------------
# LOAD MUJOCO MODEL
# -------------------------------

model = mujoco.MjModel.from_xml_path(XML_PATH)
data  = mujoco.MjData(model)

print("\nScene loaded successfully")
print("Bodies:", model.nbody)
print("Joints:", model.njnt)
print("Actuators:", model.nu)

# -------------------------------
# CHECK SENSORS
# -------------------------------

touch_sensors = [
    model.sensor(i).name
    for i in range(model.nsensor)
    if model.sensor(i).type == mujoco.mjtSensor.mjSENS_TOUCH
]

print("Touch sensors:", touch_sensors)

# -------------------------------
# ACTUATORS
# -------------------------------

print("\n=== ACTUATORS ===")
for i in range(model.nu):
    print(i, model.actuator(i).name)

# пальцы: 2..19 (0–1 — запястье)
FINGER_ACTUATORS = list(range(2, 20))

# -------------------------------
# HELPERS
# -------------------------------

def get_contacts(model, data, object_keyword="clock"):
    contacts = []
    for i in range(data.ncon):
        c = data.contact[i]
        g1 = model.geom(c.geom1).name
        g2 = model.geom(c.geom2).name
        if object_keyword in g1 or object_keyword in g2:
            contacts.append((g1, g2))
    return contacts


def is_grasping(model, data, min_contacts=2):
    return len(get_contacts(model, data)) >= min_contacts


def reset_simulation(model, data, viewer):
    mujoco.mj_resetData(model, data)
    mujoco.mj_forward(model, data)
    viewer.sync()
    time.sleep(0.2)

# -------------------------------
# LAUNCH VIEWER
# -------------------------------

viewer = mujoco.viewer.launch_passive(model, data)

# -------------------------------
# GRASP TRIALS
# -------------------------------

for trial in range(NUM_TRIALS):
    print(f"\n==============================")
    print(f" TRIAL {trial + 1}/{NUM_TRIALS}")
    print(f"==============================")

    reset_simulation(model, data, viewer)

    # --- OPEN HAND ---
    for _ in range(OPEN_STEPS):
        data.ctrl[:] = 0.0
        mujoco.mj_step(model, data)
        viewer.sync()
        time.sleep(0.005)

    # --- CLOSE FINGERS ---
    for _ in range(CLOSE_STEPS):
        data.ctrl[:] = 0.0
        for idx in FINGER_ACTUATORS:
            data.ctrl[idx] = 1.0
        mujoco.mj_step(model, data)
        viewer.sync()
        time.sleep(0.005)

    print("Contacts after close:", get_contacts(model, data))
    print("Is grasping:", is_grasping(model, data))

    # --- HOLD ---
    for _ in range(HOLD_STEPS):
        for idx in FINGER_ACTUATORS:
            data.ctrl[idx] = 1.0
        mujoco.mj_step(model, data)
        viewer.sync()
        time.sleep(0.01)

    print("Final grasp check:", is_grasping(model, data))

# -------------------------------
# KEEP WINDOW OPEN
# -------------------------------

print("\nAll trials finished.")
print("MuJoCo window will stay open.")
print("Close the viewer window manually to exit.")

while viewer.is_running():
    mujoco.mj_step(model, data)
    viewer.sync()
    time.sleep(0.02)


Assets dir: c:\Users\d_v_o\anaconda3\envs\dexgrasp\lib\site-packages\gymnasium_robotics\envs\assets\hand
Clock scene XML exists: True

Scene loaded successfully
Bodies: 29
Joints: 25
Actuators: 20
Touch sensors: ['robot0:ST_Tch_fftip', 'robot0:ST_Tch_mftip', 'robot0:ST_Tch_rftip', 'robot0:ST_Tch_lftip', 'robot0:ST_Tch_thtip', 'robot0:TS_palm_b0', 'robot0:TS_palm_bl', 'robot0:TS_palm_bm', 'robot0:TS_palm_br', 'robot0:TS_palm_fl', 'robot0:TS_palm_fm', 'robot0:TS_palm_fr', 'robot0:TS_palm_b1', 'robot0:TS_ffproximal_front_left_bottom', 'robot0:TS_ffproximal_front_right_bottom', 'robot0:TS_ffproximal_front_left_top', 'robot0:TS_ffproximal_front_right_top', 'robot0:TS_ffproximal_back_left', 'robot0:TS_ffproximal_back_right', 'robot0:TS_ffproximal_tip', 'robot0:TS_ffmiddle_front_left', 'robot0:TS_ffmiddle_front_right', 'robot0:TS_ffmiddle_back_left', 'robot0:TS_ffmiddle_back_right', 'robot0:TS_ffmiddle_tip', 'robot0:TS_fftip_front_left', 'robot0:TS_fftip_front_right', 'robot0:TS_fftip_back_le

: 

In [ ]:
# ниже код с использованием сенсоров

In [1]:
# ============================================================
# Shadow Dexterous Hand + Crosley Alarm Clock
# Contact-aware grasp with tactile sensors (5 trials)
# ============================================================

import os
import time
import numpy as np
import mujoco
import mujoco.viewer
import gymnasium_robotics

# -------------------------------
# CONFIG
# -------------------------------

ASSETS_DIR = os.path.join(
    os.path.dirname(gymnasium_robotics.__file__),
    "envs", "assets", "hand"
)

XML_PATH = os.path.join(ASSETS_DIR, "hand_manipulate_clock.xml")

NUM_TRIALS  = 5
OPEN_STEPS  = 40
CLOSE_STEPS = 120
HOLD_STEPS  = 200

print("Assets dir:", ASSETS_DIR)
print("Clock scene XML exists:", os.path.exists(XML_PATH))

# -------------------------------
# LOAD MODEL
# -------------------------------

model = mujoco.MjModel.from_xml_path(XML_PATH)
data  = mujoco.MjData(model)

print("\nScene loaded successfully")
print("Bodies:", model.nbody)
print("Joints:", model.njnt)
print("Actuators:", model.nu)

# -------------------------------
# ACTUATORS
# -------------------------------

print("\n=== ACTUATORS ===")
for i in range(model.nu):
    print(i, model.actuator(i).name)

# пальцы (0–1 — запястье)
FINGER_ACTUATORS = list(range(2, 20))

# -------------------------------
# TOUCH SENSOR HELPERS
# -------------------------------

def read_active_touch_sensors(model, data, threshold=0.01):
    active = []
    offset = 0

    for i in range(model.nsensor):
        s = model.sensor(i)
        dim = s.dim
        values = data.sensordata[offset:offset + dim]

        if s.type == mujoco.mjtSensor.mjSENS_TOUCH:
            if np.any(values > threshold):
                active.append((s.name, float(values[0])))

        offset += dim

    return active


def reset_simulation(model, data, viewer):
    mujoco.mj_resetData(model, data)
    mujoco.mj_forward(model, data)
    viewer.sync()
    time.sleep(0.2)

# -------------------------------
# VIEWER
# -------------------------------

viewer = mujoco.viewer.launch_passive(model, data)

# -------------------------------
# GRASP TRIALS
# -------------------------------

for trial in range(NUM_TRIALS):
    print(f"\n==============================")
    print(f" TRIAL {trial + 1}/{NUM_TRIALS}")
    print(f"==============================")

    reset_simulation(model, data, viewer)

    # --- OPEN HAND ---
    for _ in range(OPEN_STEPS):
        data.ctrl[:] = 0.0
        mujoco.mj_step(model, data)
        viewer.sync()
        time.sleep(0.005)

    # --- CLOSE WITH TACTILE FEEDBACK ---
    for _ in range(CLOSE_STEPS):
        active = read_active_touch_sensors(model, data)

        for idx in FINGER_ACTUATORS:
            if len(active) == 0:
                data.ctrl[idx] = 1.0   # закрываем
            else:
                data.ctrl[idx] = 0.2   # удерживаем, не дожимаем

        mujoco.mj_step(model, data)
        viewer.sync()
        time.sleep(0.005)

    active = read_active_touch_sensors(model, data)
    print("Active touch sensors after close:", len(active))
    for name, val in active[:8]:
        print(f"  {name}: {val:.4f}")

    # --- HOLD ---
    for _ in range(HOLD_STEPS):
        active = read_active_touch_sensors(model, data)

        for idx in FINGER_ACTUATORS:
            data.ctrl[idx] = 0.2 if len(active) > 0 else 0.5

        mujoco.mj_step(model, data)
        viewer.sync()
        time.sleep(0.01)

    active = read_active_touch_sensors(model, data)
    print("Final active sensors:", len(active))

# -------------------------------
# KEEP WINDOW OPEN
# -------------------------------

print("\nAll trials finished.")
print("MuJoCo window will stay open.")
print("Close the viewer window manually to exit.")

while viewer.is_running():
    mujoco.mj_step(model, data)
    viewer.sync()
    time.sleep(0.02)


Assets dir: c:\Users\d_v_o\anaconda3\envs\dexgrasp\lib\site-packages\gymnasium_robotics\envs\assets\hand
Clock scene XML exists: True

Scene loaded successfully
Bodies: 29
Joints: 25
Actuators: 20

=== ACTUATORS ===
0 robot0:A_WRJ1
1 robot0:A_WRJ0
2 robot0:A_FFJ3
3 robot0:A_FFJ2
4 robot0:A_FFJ1
5 robot0:A_MFJ3
6 robot0:A_MFJ2
7 robot0:A_MFJ1
8 robot0:A_RFJ3
9 robot0:A_RFJ2
10 robot0:A_RFJ1
11 robot0:A_LFJ4
12 robot0:A_LFJ3
13 robot0:A_LFJ2
14 robot0:A_LFJ1
15 robot0:A_THJ4
16 robot0:A_THJ3
17 robot0:A_THJ2
18 robot0:A_THJ1
19 robot0:A_THJ0

 TRIAL 1/5


TypeError: only integer scalar arrays can be converted to a scalar index